# Fix Me

This code is broken, and when running produces an OK model (r2 of ~0.53)

- Fix the code
- Improve the model

In [9]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [61]:
data = pd.read_csv('./data/rossman_reduced.csv')
extra_data = pd.read_csv('./data/rossman-extra.csv')

In [62]:
data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,4,45,2011-01-07,31.41,False
1,24,50,2012-08-03,925.00,False
2,44,81,2011-03-25,7418.05,False
3,4,38,2012-03-09,86432.14,False
4,11,80,2010-10-29,12184.46,False


In [63]:
extra_data.drop('IsHoliday',axis=1,inplace=True)  #pull
data = data.merge(extra_data, on=['Store','Date'])


In [64]:
data.shape

(30000, 14)

In [65]:
data.fillna(0)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,4,45,2011-01-07,31.41,False,39.34,2.980,0.00,0.00,0.00,0.00,0.00,127.191774,6.510
1,4,11,2011-01-07,29259.99,False,39.34,2.980,0.00,0.00,0.00,0.00,0.00,127.191774,6.510
2,4,8,2011-01-07,64561.77,False,39.34,2.980,0.00,0.00,0.00,0.00,0.00,127.191774,6.510
3,4,97,2011-01-07,37019.67,False,39.34,2.980,0.00,0.00,0.00,0.00,0.00,127.191774,6.510
4,4,26,2011-01-07,11334.47,False,39.34,2.980,0.00,0.00,0.00,0.00,0.00,127.191774,6.510
5,4,24,2011-01-07,7391.66,False,39.34,2.980,0.00,0.00,0.00,0.00,0.00,127.191774,6.510
6,24,50,2012-08-03,925.00,False,73.26,3.819,11887.54,557.10,138.39,9345.76,2781.67,138.173581,8.953
7,24,14,2012-08-03,17817.43,False,73.26,3.819,11887.54,557.10,138.39,9345.76,2781.67,138.173581,8.953
8,44,81,2011-03-25,7418.05,False,42.38,3.435,0.00,0.00,0.00,0.00,0.00,128.616064,7.224
9,44,93,2011-03-25,9557.62,False,42.38,3.435,0.00,0.00,0.00,0.00,0.00,128.616064,7.224


In [66]:
data = pd.get_dummies(data,prefix='dept',columns=['Dept'])

In [67]:
mdf = data.drop(['Date'],axis=1)

In [68]:
mdf.head()

,Store,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,dept_90,dept_91,dept_92,dept_93,dept_94,dept_95,dept_96,dept_97,dept_98,dept_99
0,4,31.41,False,39.34,2.98,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,4,29259.99,False,39.34,2.98,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,4,64561.77,False,39.34,2.98,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,4,37019.67,False,39.34,2.98,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
4,4,11334.47,False,39.34,2.98,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [69]:
mdf.shape

(30000, 92)

In [70]:
X = mdf.drop('Weekly_Sales',axis=1)
y = mdf['Weekly_Sales']

In [71]:
kf = KFold(n_splits=4,shuffle=True)

In [72]:
lr_r2 = []
l1_r2 = []
l2_r2 = []

lr_rmse = []
l1_rmse = []
l2_rmse = []

for tr,te in kf.split(X,y):
#     print("Fold")
    
    lr = LinearRegression()
    l1 = Lasso()
    l2 = Ridge()
    
    X_tr,X_te = X[tr] ,X[te]
    y_tr,y_te = y[tr] ,y[te]
    
    scale = StandardScaler()
    scale.fit(X)
    
    X_tr = scale.fit_transform(X_tr)
    X_te = scale.fit_transform(X_te)
    
#     lr.fit(X_tr,y_tr)
#     print("L1")
    l1.fit(X_tr,y_tr)
#     print("L2")
    l2.fit(X_tr,y_tr)
    
#     lr_r2.append(r2_score(y_te,lr.predict(X_te)))
    l1_r2.append(r2_score(y_te,l1.predict(X_te)))
    l2_r2.append(r2_score(y_te,l1.predict(X_te)))
    
    l1_rmse.append(np.sqrt(mean_squared_error(y_te,l1.predict(X_te))))
    
    
    
    
    
    

KeyError: '[    1     3     4 ... 29997 29998 29999] not in index'